In [4]:
# naver api
client_id = '1gd7nt7i7v'
client_pw = 'MyCsg43hgzOpddIY7CL3I5LEniZMwOyOYUrHjdXx'

# res_type
res_type = 'json'   # 'json' or 'xml'

if res_type == 'json':
    api_url = 'https://openapi.naver.com/v1/map/geocode?query='     # json
else:
    api_url = 'https://openapi.naver.com/v1/map/geocode.xml?query='     # xml

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 주소 목록 파일 (.xlsx)
#data = pd.read_excel('list_of_address.xlsx', usecols='B, C', names=['구주소', '도로명주소'])
data = pd.DataFrame(data={'도로명주소':['서울특별시 마포구 서교동 371-7']})


# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     # geographic coordinates
for add in data['도로명주소']:
    add_urlenc = parse.quote(add)   # 주소를 URL에서 사용할 수 있도록 URL Encoding
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()  # 정상이면 200 리턴
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            if res_type == 'json':
                response_body = json.loads(response_body)   # json
                if 'result' in response_body:
                    latitude = response_body['result']['items'][0]['point']['y']
                    longitude = response_body['result']['items'][0]['point']['x']
                    print("Success!")
                else:
                    print("'result' not exist!")
                    latitude = None
                    longitude = None
            else:
                response_body = BeautifulSoup(response_body, "lxml")  # xml
                latitude = [float(xy.get_text()) for xy in response_body.find_all("y")][0]
                longitude = [float(xy.get_text()) for xy in response_body.find_all("x")][0]
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

# 결과 dataframe으로 할당
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({
                              "도로명": data['도로명주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})


NameError: name 'pd' is not defined

In [14]:
#네이버 Geocoding
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import pprint

client_id = '1gd7nt7i7v'
client_pw = 'MyCsg43hgzOpddIY7CL3I5LEniZMwOyOYUrHjdXx'

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

add = '양촌나들목'

add_urlenc = parse.quote(add)   # 주소를 URL에서 사용할 수 있도록 URL Encoding
url = api_url + add_urlenc

request = Request(url)
request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
request.add_header('X-NCP-APIGW-API-KEY', client_pw)

response = urlopen(request)

response_body = response.read().decode('utf-8')

In [15]:
pprint.pprint(response_body)

'{"status":"OK","meta":{"totalCount":0,"count":0},"addresses":[],"errorMessage":""}'
